In [14]:
import pandas as pd
import requests
import json
import wikipedia
import tqdm
from datetime import datetime
import time

In [15]:
# ---------------------------------------------
# CONFIGURATION
# ---------------------------------------------
USER = "Irpen"  # Replace with the Wikipedia username
LANG = "en"  # Wikipedia language edition (e.g., 'en', 'uk', 'ru', 'fr')
DELAY = 0.5  # seconds between API calls to be polite

# Wikipedia API endpoint
API_URL = f"https://{LANG}.wikipedia.org/w/api.php"

# ---------------------------------------------
# STEP 1 — Get list of all revisions of the user's talk page
# ---------------------------------------------
def get_user_talk_revisions(user):
    page_title = f"User talk:{user}"
    params = {
        "action": "query",
        "format": "json",
        "prop": "revisions",
        "titles": page_title,
        "rvprop": "ids|timestamp|user|comment|content",
        "rvslots": "main",
        "rvlimit": "max",
    }

    revisions = []
    continue_token = {}

    print(f"Fetching talk page revisions for {page_title}...")

    while True:
        response = requests.get(API_URL, params={**params, **continue_token})
        data = response.json()

        page = next(iter(data["query"]["pages"].values()))
        if "revisions" in page:
            for rev in page["revisions"]:
                revisions.append({
                    "rev_id": rev["revid"],
                    "timestamp": rev["timestamp"],
                    "editor": rev.get("user", "Unknown"),
                    "comment": rev.get("comment", ""),
                    "content": rev.get("slots", {}).get("main", {}).get("*", "")
                })

        # Handle pagination
        if "continue" in data:
            continue_token = data["continue"]
            time.sleep(DELAY)
        else:
            break

    print(f"Retrieved {len(revisions)} revisions.")
    return pd.DataFrame(revisions)

# ---------------------------------------------
# STEP 2 — Save revisions for later analysis
# ---------------------------------------------
def save_revisions(df, user):
    df.to_csv('..datas/interim/user_talk_history.csv', index=False)
    print(f"✅ Saved to {filename}")

df_talk = get_user_talk_revisions(USER)
save_revisions(df_talk, USER)

Fetching talk page revisions for User talk:Irpen...


JSONDecodeError: Expecting value: line 1 column 1 (char 0)